In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score
import pickle

In [16]:
# Load the dataset, drop some unneccessary column to resize
data = pd.read_csv("final_merged_data.csv")
data = data.loc[:, ~data.columns.str.contains("quality_indicator")]

# Save the cleaned version
data.to_csv("final_merged_data.csv", index=False)

In [7]:
pd.set_option('display.max_columns', None)
data.describe()

,station_id,num_bikes_available,num_docks_available,lat,lon,capacity,stno,year,month,day,hour,minute,max_air_temp_quality_indicator,max_air_temperature_celsius,min_air_temp_quality_indicator,min_air_temperature_celsius,air_temp_std_quality_indicator,air_temperature_std_deviation,max_grass_temp_quality_indicator,max_grass_temperature_celsius,min_grass_temp_quality_indicator,min_grass_temperature_celsius,grass_temp_std_quality_indicator,grass_temperature_std_deviation,max_soil_temp_5cm_quality_indicator,max_soil_temperature_5cm_celsius,min_soil_temp_5cm_quality_indicator,min_soil_temperature_5cm_celsius,soil_temp_std_5cm_quality_indicator,soil_temperature_std_deviation_5cm,max_soil_temp_10cm_quality_indicator,max_soil_temperature_10cm_celsius,min_soil_temp_10cm_quality_indicator,min_soil_temperature_10cm_celsius,soil_temp_std_10cm_quality_indicator,soil_temperature_std_deviation_10cm,max_soil_temp_20cm_quality_indicator,max_soil_temperature_20cm_celsius,min_soil_temp_20cm_quality_indicator,min_soil_temperature_20cm_celsius,soil_temp_std_20cm_quality_indicator,soil_temperature_std_deviation_20cm,max_earth_temp_30cm_quality_indicator,max_earth_temperature_30cm_celsius,min_earth_temp_30cm_quality_indicator,min_earth_temperature_30cm_celsius,earth_temp_std_30cm_quality_indicator,earth_temperature_std_deviation_30cm,max_earth_temp_50cm_quality_indicator,max_earth_temperature_50cm_celsius,min_earth_temp_50cm_quality_indicator,min_earth_temperature_50cm_celsius,earth_temp_std_50cm_quality_indicator,earth_temperature_std_deviation_50cm,max_earth_temp_100cm_quality_indicator,max_earth_temperature_100cm_celsius,min_earth_temp_100cm_quality_indicator,min_earth_temperature_100cm_celsius,earth_temp_std_100cm_quality_indicator,earth_temperature_std_deviation_100cm,max_humidity_quality_indicator,max_relative_humidity_percent,min_humidity_quality_indicator,min_relative_humidity_percent,humidity_std_quality_indicator,relative_humidity_std_deviation,max_pressure_quality_indicator,max_barometric_pressure_hpa,min_pressure_quality_indicator,min_barometric_pressure_hpa,pressure_std_quality_indicator,barometric_pressure_std_deviation
count,298946.000000,298946.000000,298946.000000,298946.000000,298946.000000,298946.000000,298946.0,298946.0,298946.0,298946.000000,298946.000000,298946.000000,298946.0,298946.000000,298946.0,298946.000000,298946.0,298946.000000,298946.0,298946.000000,298946.0,298946.000000,298946.0,298946.000000,298946.0,298946.000000,298946.0,298946.000000,298946.0,298946.000000,298946.0,298946.000000,298946.0,298946.000000,298946.0,298946.000000,298946.0,298946.000000,298946.0,298946.000000,298946.0,298946.000000,298946.0,298946.000000,298946.0,298946.000000,298946.0,298946.000000,298946.0,298946.000000,298946.0,298946.000000,298946.0,298946.000000,298946.0,298946.000000,298946.0,298946.000000,298946.0,298946.000000,298946.0,298946.000000,298946.0,298946.000000,298946.0,298946.000000,298946.0,298946.000000,298946.0,298946.000000,298946.0,298946.000000
mean,57.967138,12.204733,19.335198,53.345521,-6.263842,31.979003,175.0,2024.0,12.0,15.717484,11.522128,24.878941,0.0,7.855169,0.0,7.716685,0.0,0.040432,0.0,6.564362,0.0,6.229473,0.0,0.100257,0.0,7.047128,0.0,7.007714,0.0,0.009456,0.0,7.159650,0.0,7.125825,0.0,0.007945,0.0,7.258614,0.0,7.225430,0.0,0.007645,0.0,7.390959,0.0,7.358199,0.0,0.007551,0.0,7.995048,0.0,7.964415,0.0,0.007055,0.0,9.437616,0.0,9.406651,0.0,0.007106,0.0,85.021124,0.0,83.548719,0.0,0.377589,0.0,1014.855870,0.0,1014.689671,0.0,0.045488
std,33.958176,9.761814,11.001506,0.007538,0.017855,7.459101,0.0,0.0,0.0,8.896832,6.280459,17.112726,0.0,3.123094,0.0,3.136997,0.0,0.038731,0.0,3.901582,0.0,3.946463,0.0,0.105303,0.0,1.508867,0.0,1.509558,0.0,0.003023,0.0,1.232544,0.0,1.233239,0.0,0.001651,0.0,0.975946,0.0,0.976103,0.0,0.001110,0.0,0.876321,0.0,0.876168,0.0,0.001084,0.0,0.537950,0.0,0.537683,0.0,0.000938,0.0,0.280474,0.0,0.280232,0.0,0.000920,0.0,7.867297,0.0,8.160979,0.0,0.232676,0.0,11.823993,0.0,11.894264,0.0,0.038661
m

In [12]:
#PREPARE DATA

# Handle missing values (drop rows with NaN in lagged features)
data.dropna(inplace=True)
# Calculate avg for weather:
data['temperature'] = (data['max_air_temperature_celsius'] + data['min_air_temperature_celsius']) / 2
data['humidity'] = (data['max_relative_humidity_percent'] + data['min_relative_humidity_percent']) / 2
data['pressure'] = (data['max_barometric_pressure_hpa'] + data['min_barometric_pressure_hpa']) / 2
#Calculate day of week:
data['date'] = pd.to_datetime(data[['year', 'month', 'day']])
data['day_of_week'] = data['date'].dt.weekday

# Define features and target
features = ['station_id','temperature', 'humidity', 'pressure', 'hour', 'day_of_week']
target = 'num_bikes_available'

X = data[features]
y = data[target]

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [13]:
#TRAINING 

# Train a linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"Mean Absolute Error: {mae}")
print(f"R² Score: {r2}")

# Display model coefficients
print("\nModel Coefficients:")
for feature, coef in zip(features, model.coef_):
    print(f"{feature}: {coef}")
print(f"Intercept: {model.intercept_}")

# Save the model to a file
#model_filename = "bike_availability_model.joblib"
#joblib.dump(model, model_filename)
#print(f"Model saved to {model_filename}")

# Save the model to a .pkl file
model_filename = "bike_availability_model.pkl"
with open(model_filename, "wb") as file:
    pickle.dump(model, file)

print(f"Model saved to {model_filename}")

Mean Absolute Error: 8.142427565204255
R² Score: 5.569462168464323e-05

Model Coefficients:
station_id: -0.00023485030364140392
temperature: 0.008470891436559044
humidity: 0.011877529633333939
pressure: -0.00013009919724920897
hour: -0.005268656137795134
day_of_week: 0.051598563188169286
Intercept: 11.220851745111297
Model saved to bike_availability_model.pkl


In [30]:
import pandas as pd
import pickle

# Load the saved model
with open("bike_availability_model.pkl", "rb") as file:
    model = pickle.load(file)

# Define new input data for prediction
new_data = pd.DataFrame({
    'station_id': [31],
    'temperature': [50],
    'humidity': [80],
    'pressure': [2000],
    'hour': [10],
    'day_of_week': [2]  # Example: 0 = Monday, 1 = Tuesday, etc.
})

# Make prediction
prediction = model.predict(new_data)
# Output prediction
print(f"Predicted number of available bikes: {prediction[0]}")



Predicted number of available bikes: 12.37763049869305
